# Explore here

In [1]:
# Your code here
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split 
from sqlalchemy import create_engine, text, Column, Integer, String
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import *
import os 
from dotenv import load_dotenv
from sqlalchemy import create_engine
import sqlite3
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [2]:
load_dotenv()

True

In [3]:
urlmovies = "https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv"
urlcreditos = "https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv"
movies = pd.read_csv(urlmovies)
creditos = pd.read_csv(urlcreditos)

In [4]:
movies = pd.DataFrame(movies)
creditos = pd.DataFrame(creditos)

In [5]:
movies.head(2)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [6]:
creditos.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [7]:
movies.tail(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4801,0,[],http://shanghaicalling.com/,126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-05-03,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7
4802,0,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,25975,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...",en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,"[{""name"": ""rusty bear entertainment"", ""id"": 87...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2005-08-05,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16


In [8]:
creditos.tail(2)

,movie_id,title,cast,crew
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."
4802,25975,My Date with Drew,"[{""cast_id"": 3, ""character"": ""Herself"", ""credi...","[{""credit_id"": ""58ce021b9251415a390165d9"", ""de..."


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

### Paso 2: Creación de una base de datos
Crea una base de datos para almacenar los dos DataFrames en tablas distintas. 
A continuación, une las dos tablas con SQL (e intégralo con Python) para generar una tercera tabla que contenga información de ambas unificada. 
La clave a través de la cual se puede hacer la unión es el título de la película (`titulo`).

In [10]:
conn = sqlite3.connect(os.getenv('DATABASE_URL'))

In [11]:

movies.to_sql("movies_table", conn, if_exists = "replace", index = False)
creditos.to_sql("credits_table", conn, if_exists = "replace", index = False)

4803

In [12]:
conn = sqlite3.connect(os.getenv('DATABASE_URL'))
query = """
    SELECT *
    FROM movies_table
    INNER JOIN credits_table
    ON movies_table.title = credits_table.title;
"""

total_data = pd.read_sql_query(query, conn)
conn.close()

total_data = total_data.loc[:, ~total_data.columns.duplicated()]
total_data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [13]:
# limpia la tabla generada y deja solo las siguientes columnas
total_data = total_data[['movie_id','title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
total_data.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [14]:
def get_movie_recommendations(movie_title):
    movie_index = total_data[total_data['title'] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(total_data['title'][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]


### Paso 3: Transforma los datos
De cada uno de los JSONs, selecciona el atributo name y reemplaza las columnas genres y keywords.

In [15]:
total_data['genres'] = total_data['genres'].apply(json.loads)    # convierto Json en dicts
total_data['genres'] = total_data['genres'].apply(lambda x: [d['name'] for d in x])     # por cada dict in genres tomo d['name']
total_data['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4809, dtype: object

In [16]:
total_data['keywords'] = total_data['keywords'].apply(json.loads)           # convierto Json en dicts
total_data['keywords'] = total_data['keywords'].apply(lambda x: [d['name'] for d in x])     # por cada dict in genres tomo d['name']
total_data['keywords']

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4809, dtype: object

In [17]:
total_data['cast'] = total_data['cast'].apply(json.loads)   # convierto en json los dic
total_data['cast'] = total_data['cast'].apply(lambda x: [d['name'] for d in x[0:3]])# por cada dict in genres tomo d['name'] y en este caso los primeros 3 valores [0:3]
total_data['cast']

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4809, dtype: object

In [18]:
total_data['overview'] = total_data['overview'].apply(lambda x: [x])
total_data['overview'].iloc[0]

['In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.']

In [19]:
total_data.crew[0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [20]:
total_data['crew'] = total_data['crew'].apply(json.loads)                          # convierto Json en dicts
total_data['crew'] = total_data['crew'].apply(lambda x: [d['name'] for d in x])    
total_data['crew']

0       [Stephen E. Rivkin, Rick Carter, Christopher B...
1       [Dariusz Wolski, Gore Verbinski, Jerry Bruckhe...
2       [Thomas Newman, Sam Mendes, Anna Pinnock, John...
3       [Hans Zimmer, Charles Roven, Christopher Nolan...
4       [Andrew Stanton, Andrew Stanton, John Lasseter...
                              ...                        
4804    [Robert Rodriguez, Robert Rodriguez, Robert Ro...
4805    [Edward Burns, Edward Burns, Edward Burns, Wil...
4806    [Carla Hetland, Harvey Kahn, Adam Sliwinski, M...
4807                           [Daniel Hsia, Daniel Hsia]
4808    [Clark Peterson, Andrew Reimer, Brian Herzling...
Name: crew, Length: 4809, dtype: object

In [21]:
# Generamos una columna nueva con los contenidos de genres, cast, crew y keywords.
total_data['tags'] = total_data['genres'] + total_data['cast'] + total_data['crew'] +  total_data['keywords'] + total_data['overview']

In [22]:
total_data['tags'][0]

['Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen E. Rivkin',
 'Rick Carter',
 'Christopher Boyes',
 'Christopher Boyes',
 'Mali Finn',
 'James Horner',
 'James Cameron',
 'James Cameron',
 'James Cameron',
 'James Cameron',
 'James Cameron',
 'Andrew Menzies',
 'Jill Brooks',
 'Margery Simkin',
 'Kevin Ishioka',
 'Dick Bernstein',
 'Shannon Mills',
 'Dennie Thorpe',
 'Jana Vance',
 'Deborah Lynn Scott',
 'Jon Landau',
 'Sean Haworth',
 'Kim Sinclair',
 'Kim Sinclair',
 'Richard F. Mays',
 'Laeta Kalogridis',
 'Mayes C. Rubeo',
 'Mauro Fiore',
 'Scott Herbertson',
 'Woody Schultz',
 'Linda DeVetta',
 'Linda DeVetta',
 'Richard Bluck',
 'Simon Bright',
 'Richard Martin',
 'Steve R. Moore',
 'John Refoua',
 'Karl J. Martin',
 'Chiling Lin',
 'Ilram Choi',
 'Steven Quale',
 'Carla Meyer',
 'Nick Bassett',
 'Jill Cormack',
 'Andy McLaren',
 'Terry Notary',
 'Garrett Warren',
 'Jonathan Rothbart',
 'Stefan Dechant',
 

In [23]:
print(total_data['tags'].isnull().sum())

0


In [24]:
print(total_data['tags'].apply(type).value_counts())

tags
<class 'list'>    4809
Name: count, dtype: int64


In [25]:
def process_tags(tag_list):
    # Filtrar elementos None y convertir otros tipos a string
    filtered_tags = [str(tag).replace(" ", "") for tag in tag_list if tag is not None]
    return " ".join(filtered_tags)

#Aplicar la función a la columna 'tags'
total_data['tags'] = total_data['tags'].apply(process_tags)

In [26]:
total_data['tags'][0]

"Action Adventure Fantasy ScienceFiction SamWorthington ZoeSaldana SigourneyWeaver StephenE.Rivkin RickCarter ChristopherBoyes ChristopherBoyes MaliFinn JamesHorner JamesCameron JamesCameron JamesCameron JamesCameron JamesCameron AndrewMenzies JillBrooks MargerySimkin KevinIshioka DickBernstein ShannonMills DennieThorpe JanaVance DeborahLynnScott JonLandau SeanHaworth KimSinclair KimSinclair RichardF.Mays LaetaKalogridis MayesC.Rubeo MauroFiore ScottHerbertson WoodySchultz LindaDeVetta LindaDeVetta RichardBluck SimonBright RichardMartin SteveR.Moore JohnRefoua KarlJ.Martin ChilingLin IlramChoi StevenQuale CarlaMeyer NickBassett JillCormack AndyMcLaren TerryNotary GarrettWarren JonathanRothbart StefanDechant ToddCherniawsky MirandaRivers RobertStromberg JohnHarding RobertoDeAngelis MikeSmithson AlainLalanne LucasSalton JanaceTashjian StephenRosenbaum FrankieKarena LisaLovaas JonathanFawkner RobertBavin AnthonyAlmaraz CarolynM.Fenton BethKoenigsberg SamPage TexKadonaga KimFoscato TammyS.

In [27]:
total_data.drop(columns = ["genres","cast","crew","keywords","overview"])

,movie_id,title,tags
0,19995,Avatar,Action Adventure Fantasy ScienceFiction SamWor...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action JohnnyDepp OrlandoBlo...
2,206647,Spectre,Action Adventure Crime DanielCraig ChristophWa...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller ChristianBale Mich...
4,49529,John Carter,Action Adventure ScienceFiction TaylorKitsch L...
...,...,...,...
4804,9367,El Mariachi,Action Crime Thriller CarlosGallardo JaimedeHo...
4805,72766,Newlyweds,Comedy Romance EdwardBurns KerryBishé MarshaDi...
4806,231617,"Signed, Sealed, Delivered",Comedy Drama Romance TVMovie EricMabius Kristi...
4807,126186,Shanghai Calling,DanielHenney ElizaCoupe BillPaxton DanielHsia ...


## Paso 4: Construye un KNN
Para resolver este problema crearemos nosotros nuestro propio KNN. Lo primero de todo es vectorizar el texto siguiendo los mismos pasos que aprendiste en la lección anterior.

In [28]:
#Comprobamos los vecinos cercanos
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(total_data["tags"])
tfidf_matrix.shape

(4809, 99820)

In [1]:
def get_movie_recommendations(movie_title):
    # Encuentra el índice de la película en el DataFrame
    movie_index = total_data[total_data['title'] == movie_title].index[0]
    # obtener las películas similares a traves del indice 
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    # Crea una lista de películas similares con sus distancias
    similar_movies = [(total_data['title'].iloc[i], distances[0][j]) for j, i in enumerate(indices[0])]
    # Retorna todas las películas similares
    return similar_movies[1:]

In [43]:
model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")
model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [44]:
get_movie_recommendations('Avatar')

[('True Lies', np.float64(0.8584154625150732)),
 ('Titanic', np.float64(0.8645418586578005)),
 ('Aliens', np.float64(0.8990094935665087)),
 ('Terminator 2: Judgment Day', np.float64(0.9030925456404042)),
 ('The Hobbit: The Desolation of Smaug', np.float64(0.9334319546787581))]